In [2]:
# Library
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

import re
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [21]:
token_df = pd.read_csv('./Data/Preprocess_Data/최종추출본.csv')

In [13]:
# token_df['year'] = token_df['year'].astype('str')

In [22]:
token_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2267163 entries, 0 to 2267162
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   list    object
 1   year    int64 
dtypes: int64(1), object(1)
memory usage: 34.6+ MB


In [ ]:
t = token_df.copy()
# year 칼럼의 뒤의 두 자리 추출
t['year_suffix'] = t['year'].astype(str).str[-2:]
# list 칼럼의 단어들을 분리하여 리스트로 변환
word_lists = t['list'].str.split()

In [ ]:
new_word_lists = []
for words, suffix in tqdm(zip(word_lists, t['year_suffix']), total=len(t)):
    new_words = [str(suffix) + '_' + word for word in words]
    new_word_lists.append(new_words)

In [20]:
new_word_lists = []
for words, suffix in tqdm(zip(word_lists, t['year_suffix']), total=len(t)):
    if isinstance(suffix, float):
        suffix = str(int(suffix))[-2:]  # float 형식을 int로 변환하고 마지막 두 자리 숫자를 추출
    new_words = [suffix + '_' + word for word in words]
    new_word_lists.append(new_words)

t['tokens'] = pd.Series([' '.join(words) for words in tqdm(new_word_lists)])

t = t.drop('year_suffix', axis=1)

 70%|█████████████████████▊         | 1592053/2267163 [07:28<03:10, 3546.37it/s]


TypeError: 'float' object is not iterable

In [6]:
t.to_csv('Data/Preprocess_Data/word_token+year.csv', encoding='utf-8-sig', index=False)

In [5]:
# 2020년 데이터만
t_2023 = t[t['year']==2023]
t_2023.reset_index(drop=True, inplace=True)

In [16]:
len(t_2023)

36158

In [18]:
t_2023.head()

,tokens,year
0,23_numerical_weather_prediction_model 23_impor...,2023
1,23_great 23_number 23_virtual_museum 23_exhibi...,2023
2,23_processing 23_locally 23_harvested 23_data ...,2023
3,23_seismic 23_p 23_wave 23_first 23_arrival 23...,2023
4,23_research 23_ha 23_shown 23_spatial_percepti...,2023


In [7]:
# 텍스트 데이터 전처리 및 토큰화
t_2023['tokens2'] = t_2023['tokens'].astype(str).apply(lambda x: word_tokenize(x))
t_2023['tokens2'] = t_2023['tokens2'].apply(lambda tokens: [token for token in tokens if token != ','])

Processing sentences: 100%|██████████| 36158/36158 [00:00<00:00, 4016995.39it/s]


In [8]:

sentences = t_2023['tokens2'].tolist()
sentences = [sentence for sentence in tqdm(sentences, desc="Processing sentences")]

Processing sentences: 100%|██████████| 36158/36158 [00:00<00:00, 4908173.21it/s]


In [9]:
sentences

[['23_numerical_weather_prediction_model',
  '23_important',
  '23_instrument',
  '23_predict',
  '23_future',
  '23_weather',
  '23_provenance',
  '23_information',
  '23_central',
  '23_importance',
  '23_detecting',
  '23_unexpected',
  '23_event',
  '23_may',
  '23_develop',
  '23_long',
  '23_course',
  '23_model_execution',
  '23_besides',
  '23_need',
  '23_share',
  '23_scientific_data',
  '23_result',
  '23_researcher',
  '23_also',
  '23_highlight',
  '23_importance',
  '23_data_quality',
  '23_reliability',
  '23_weather',
  '23_research',
  '23_forecasting_model',
  '23_open_source',
  '23_nwp',
  '23_model',
  '23_study',
  '23_propose',
  '23_methodology',
  '23_tracking',
  '23_wrf',
  '23_model',
  '23_generating',
  '23_storing',
  '23_analyzing',
  '23_provenance',
  '23_implement',
  '23_proposed',
  '23_methodologywith',
  '23_machine_learning_based',
  '23_parser',
  '23_utilizes',
  '23_classification_algorithm',
  '23_extract',
  '23_provenance',
  '23_informatio

In [10]:
# sentences_flat에서 19년도 단어가 들어간 데이터만 찾기
sentences_flat = [word for sentence in sentences for word in sentence]

In [11]:
sentences_flat

['23_numerical_weather_prediction_model',
 '23_important',
 '23_instrument',
 '23_predict',
 '23_future',
 '23_weather',
 '23_provenance',
 '23_information',
 '23_central',
 '23_importance',
 '23_detecting',
 '23_unexpected',
 '23_event',
 '23_may',
 '23_develop',
 '23_long',
 '23_course',
 '23_model_execution',
 '23_besides',
 '23_need',
 '23_share',
 '23_scientific_data',
 '23_result',
 '23_researcher',
 '23_also',
 '23_highlight',
 '23_importance',
 '23_data_quality',
 '23_reliability',
 '23_weather',
 '23_research',
 '23_forecasting_model',
 '23_open_source',
 '23_nwp',
 '23_model',
 '23_study',
 '23_propose',
 '23_methodology',
 '23_tracking',
 '23_wrf',
 '23_model',
 '23_generating',
 '23_storing',
 '23_analyzing',
 '23_provenance',
 '23_implement',
 '23_proposed',
 '23_methodologywith',
 '23_machine_learning_based',
 '23_parser',
 '23_utilizes',
 '23_classification_algorithm',
 '23_extract',
 '23_provenance',
 '23_information',
 '23_proposed',
 '23_approach',
 '23_enables',
 '23

In [ ]:
# 주어진 단어들에 대한 Word2Vec 결과 벡터 추출
word_vecs = [word_vectors[word] for word in sentences_flat]

# 모든 단어 쌍의 코사인 유사도 계산
similarity_matrix = cosine_similarity(word_vecs)

# 결과를 DataFrame으로 변환
similarity_df = pd.DataFrame(similarity_matrix, index=sentences_flat, columns=sentences_flat)

In [ ]:
# 유사성 임계값 설정
similarity_threshold = 0.13

# 주어진 단어들에 대한 Word2Vec 결과 벡터 추출
word_vecs = [word_vectors[word] for word in sentences_flat if word in word_vectors]

# 각 커뮤니티의 중심점을 찾기
centroids = []
for community in communities:
    community_vectors = [word_vectors[word] for word in community if word in word_vectors]
    if len(community_vectors) > 0:
        community_centroid = sum(community_vectors) / len(community_vectors)
        centroids.append(community_centroid)

# 커뮤니티 중심점을 기반으로 새 그래프 생성
G_centroids = nx.Graph()

# 중심점 노드 추가
for i, centroid in enumerate(community_centroids):
    G_centroids.add_node(centroid, label=f"Community {i+1}")

# 코사인 유사도를 사용하여 중심점 간의 유사도 계산 및 연결
G_centroids = nx.Graph()
for i in range(len(centroids)):
    for j in range(i+1, len(centroids)):
        centroid_i = centroids[i]
        centroid_j = centroids[j]
        similarity = cosine_similarity([centroid_i], [centroid_j])[0][0]
        if similarity > similarity_threshold:
            G_centroids.add_edge(f"Centroid {i+1}", f"Centroid {j+1}", weight=similarity)

# 그래프 그리기
fig, ax = plt.subplots(figsize=(10, 7))
node_labels = nx.get_node_attributes(G_centroids, 'label')
nx.draw(G_centroids, labels=node_labels, ax=ax)
plt.title('Paper Abstract Network with Communities')
plt.show()

In [ ]:
# 유사성 임계값 설정
similarity_threshold = 0.13

# 각 커뮤니티의 중심점을 찾기
centrality = nx.betweenness_centrality(G)
community_centroids = [max(community, key=lambda x: centrality[x]) for community in communities]

# 커뮤니티 중심점을 기반으로 새 그래프 생성
G_centroids = nx.Graph()

# 중심점 노드 추가
for i, centroid in enumerate(community_centroids):
    G_centroids.add_node(centroid, label=f"Community {i+1}")

# 중심점 간의 유사도 계산
for i in range(len(community_centroids)):
    for j in range(i+1, len(community_centroids)):
        vec_i = vectorizer.transform([df[df['title'] == community_centroids[i]]['abstract'].values[0]])
        vec_j = vectorizer.transform([df[df['title'] == community_centroids[j]]['abstract'].values[0]])
        similarity = cosine_similarity(vec_i, vec_j)[0][0]

        # 유사도가 임계값 이상인 경우만 연결
        if similarity > similarity_threshold:
            G_centroids.add_edge(community_centroids[i], community_centroids[j], weight=similarity)

# 그래프 그리기
fig, ax = plt.subplots(figsize=(10, 7))
node_labels = nx.get_node_attributes(G_centroids, 'label')
nx.draw(G_centroids, labels=node_labels, ax=ax)
plt.title('Paper Abstract Network with Communities')
plt.show()

In [4]:
model = Word2Vec.load('Data/Preprocess_Data/year_word2vec.model')
word_vectors = KeyedVectors.load("Data/Preprocess_Data/year_word_vectors.kv")

In [17]:
word_vectors['23_3d']

array([-0.93883795,  0.8702791 , -0.46882793,  1.7246755 , -0.8193301 ,
        0.9934894 , -0.26157996,  0.5903084 , -1.4105037 , -0.8141003 ,
       -1.1219885 ,  0.0975655 , -0.741658  ,  1.3774621 ,  1.5476637 ,
       -0.6819689 , -1.907861  ,  0.5611645 ,  2.8857677 , -0.5770614 ,
        0.10597871,  1.7821556 ,  0.27356502, -0.15457615,  0.99970806,
        1.1283612 , -1.1640093 ,  0.24077725,  0.31795156,  0.5620525 ,
       -2.1021833 ,  0.93898535, -0.08460349, -0.01459267,  2.0066764 ,
       -0.02914749, -0.7029847 ,  0.85222894, -0.9882712 ,  1.4000542 ,
       -0.01609137, -0.57701606, -0.26031092, -1.1427237 , -0.05676638,
        0.31858853, -0.5801323 , -0.7727626 , -0.7112777 ,  0.33302307,
        0.50514704,  0.01409066, -2.2365885 ,  0.8775904 , -0.5543012 ,
        1.0996583 , -0.9990607 , -1.2389967 , -0.99310136,  0.28972542,
       -0.26034322,  0.40335733, -0.72999156, -0.0983801 , -0.58133197,
       -1.1497376 , -0.44267395,  1.7590141 , -1.7914195 ,  1.73

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import networkx as nx

# 각 논문을 단어 벡터의 평균값으로 표현
document_vectors = []
for tokens in t_2023['tokens']:
    words = tokens.split()
    vectors = [word_vectors[word] for word in words if word in word_vectors]
    if vectors:
        document_vector = sum(vectors) / len(vectors)
        document_vectors.append(document_vector)
    else:
        # 만약 단어 벡터가 없는 경우, 임의의 벡터를 사용하거나 건너뛸 수 있음
        document_vectors.append(None)

# 코사인 유사도를 사용하여 각 논문의 유사성 계산 (여기서 행이 한번 죽었음)
similarities = cosine_similarity(document_vectors)


In [ ]:
# 유사성 임계값 설정
similarity_threshold = 0.5

# 각 커뮤니티의 중심점을 찾기
centroids = []
for community in communities:
    community_vectors = [document_vectors[idx] for idx in community]
    community_centroid = sum(community_vectors) / len(community_vectors)
    centroids.append(community_centroid)

# 그래프 생성
G = nx.Graph()

# 중심점 간의 유사도 계산 및 연결
for i in range(len(centroids)):
    for j in range(i + 1, len(centroids)):
        centroid_i = centroids[i]
        centroid_j = centroids[j]
        similarity = cosine_similarity([centroid_i], [centroid_j])[0][0]
        if similarity > similarity_threshold:
            G.add_edge(f"Centroid {i+1}", f"Centroid {j+1}", weight=similarity)

# 그래프 그리기
fig, ax = plt.subplots(figsize=(10, 7))
nx.draw(G, with_labels=True, ax=ax)
plt.title('Paper Abstract Network with Centroid Similarities')
plt.show()